## Keras Tuner

[Keras Tuner](https://keras.io/keras_tuner/) is a high-level library for hyperparameter selection with Keras.

It allows the definition of two main components:

1. A __search space__, denoting all the possible hyperparameter configurations that we can choose;
2. A __search algorithm__, which tells us how to "move" within the search space to choose the optimal configuration.

In [ ]:
import keras as K
import keras_tuner as kt

In [ ]:
kt.__version__

### Search space API

In [ ]:
from keras_tuner import HyperParameters

We instantiate an empty search space by invoking the class `HyperParameters`

In [ ]:
hp = HyperParameters()

`khp.Choice` defines the space for "categorical" hyperparameters

In [ ]:
help(hp.Choice)
print(hp.Choice(name='activation', values=['tanh', 'relu']))

`hp.Boolean` defines the space for boolean hyperparameters

In [ ]:
help(hp.Boolean)
print(hp.Boolean(name='use_bias'))

`hp.Fixed` defines a fixed hyperparameter

In [ ]:
help(hp.Fixed)
print(hp.Fixed(name='batch_size', value=32))

`hp.Float` allows choosing float values within an interval.

In [ ]:
help(hp.Float)
print(hp.Float(name='learning_rate', min_value=0.0001, max_value=0.1, sampling='log'))

`hp.Int` same as above, for integers.

In [ ]:
help(hp.Int)
print(hp.Int(name='units', min_value=1, max_value=5))

Now ``hp`` defines the space of hyperparameters we can choose from.

In [ ]:
print(hp.get('units'))
print(hp.get('learning_rate'))
print(hp.get('batch_size'))
print(hp.get('use_bias'))
print(hp.get('activation'))

### Search Algorithm

An example of search algorithm is Random Search. Keras Tuner offers a class ``RandomSearch``, whose constructor needs:

1. A ``build`` function which instantiates the model given the hp configuration;
2. An objective function to assess the quality of the models;
3. A number of trials.

In [ ]:
def build_model(hp):
    model = K.Sequential()
    model.add(K.layers.Flatten())
    model.add(K.layers.Rescaling(scale=1/255.))

    model.add(K.layers.Dense(
        units=hp.Choice('units', [50, 100, 1000]),
        activation='relu'))
    model.add(K.layers.Dense(10, activation='softmax', use_bias=hp.Boolean('use_bias')))

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5
)

In [ ]:
from keras.datasets import mnist, boston_housing
from sklearn.model_selection import train_test_split

In [ ]:
(train_X, train_y), (test_X, test_y) = mnist.load_data(path='ds')

In [ ]:
train_X, eval_X, train_y, eval_y = train_test_split(
    train_X, train_y,
    test_size=0.15,
    shuffle=True,
    stratify=train_y
)

The search function gets exactly the same parameters as `model.fit`.

In [ ]:
tuner.search(train_X, train_y, epochs=5, batch_size=1000, validation_data=(eval_X, eval_y))

In [ ]:
best_model = tuner.get_best_models()[0]
prediction = best_model(train_X[:10])
best_model.summary()

In [ ]:
tuner.get_best_hyperparameters()[0].get_config()['values']

# Your Turn!

Try to implement a model selection with Keras Tuner on the usual Boston Housing dataset.

In [ ]:
(h_train_X, h_train_y), (h_test_X, h_test_y) = kds.boston_housing.load_data()

In [ ]:
import random
import numpy as np

h_train = np.concatenate([h_train_X, h_train_y[:, np.newaxis]], axis=1)
random.seed(42)
random.shuffle(h_train)

h_train, h_eval = h_train[75:], h_train[:75]
h_train_X, h_train_y = h_train[:, :-1], h_train[:, -1]
h_eval_X, h_eval_y = h_eval[:, :-1], h_eval[:, -1]

In [ ]:
(h_train_X.shape, h_train_y.shape), (h_eval_X.shape, h_eval_y.shape)